In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
import dateutil.parser
from numpy import random
random.seed(123)

In [2]:
elec_path = '../data/processed/temp_open_utc_complete.csv'

In [3]:
meta_path = '../data/raw/meta_open.csv'

In [4]:
weather_path = '../data/external/weather/weather1.csv'

In [5]:
def cutoff_minute(dt):
    '''
    INPUT
    a datetime object has year, month, day, hour, and minute
    OUTPUT
    a datetime object has year, month, day, and hour
    '''
    year = dt.year
    month = dt.month
    day = dt.day
    hour = dt.hour
    return datetime.datetime(year, month, day, hour)

def make_dataset(elec_path, meta_path, weather_path, weather_file, industry):
    '''
    INPUT
    elec_path: path to timeseries data of electricity meter (temp_open_utc_complete.csv)
    meta: path to meta data table (meta_open.csv)
    weather: path to weather data table (weatherX.csv)
    weather_file: name of the weather file
    industry: name of undustry to focus on
    OUTPUT
    A dataframe in which each record represents a building at a certain time
    columns =[
    building_name: name of the building from meta, str
    month: one-hot coded
    day: from elec, int
    day_of_the_week: one-hot coded
    hour: from elec, hour from weather is converted to the nearest :00, int
    area: from meta, float
    primary_space_usage: from meta (primaryspaceuse_abbrev), one-hot coded
    electricity: from elec
    temperature: from weather
    ]
    ------------------------------------------------------------------------------------------------------------------------------------------
    comment:
    -humidity is sometimes missing in weather table
    '''
    #read tables
    elec = pd.read_csv(elec_path)
    meta = pd.read_csv(meta_path)
    weather = pd.read_csv(weather_path)
    #set 'uid' as index in meta
    meta = meta.set_index('uid')
    #parse date
    weather['timestamp'] = weather['timestamp'].apply(dateutil.parser.parse)
    elec['timestamp'] = elec['timestamp'].apply(dateutil.parser.parse)
    #construct the dataframe to return
    buildings = list(meta[(meta['newweatherfilename'] == weather_file) & (meta['industry']==industry)].index) #name of the buildings
    df = pd.DataFrame(columns=['building_name', 'timestamp', 'electricity', 'area', 'primary_space_usage']) #empty dataframe with 3 columns
    for building in buildings:
        subdf = elec[['timestamp', building]]
        subdf.columns = ['timestamp', 'electricity']
        subdf['building_name'] = building
        subdf['area'] = meta.loc[building, 'sqm']
        subdf['primary_space_usage'] = meta.loc[building, 'primaryspaceuse_abbrev']
        df = pd.concat([df, subdf], axis=0, ignore_index=True)
    #df has 'building_name', timestamp, electricity meter, area, primary space usage
    print('OK1')
    weather['rounded_timestamp'] = weather['timestamp'].apply(cutoff_minute) #cutoff_minute is implemented separately
    weather = weather.groupby('rounded_timestamp').first() #only the first observation in each hour is taken
    weather = weather['TemperatureC'] #only need temperature column
    print('OK2')
    df['timestamp'] = df['timestamp'].apply(cutoff_minute)#drop time zone info
    df = df.join(weather, on='timestamp', how='inner', lsuffix='elec', rsuffix='weather') #join temperature data from weather table
    #df = add_month_day_hour(df) #to do
    #df = add_day_of_the_week()
    return df.reset_index()

In [38]:
def make_dataset(elec_path, meta_path, weather_path, weather_file, industry):
    '''
    INPUT
    elec_path: path to timeseries data of electricity meter (temp_open_utc_complete.csv)
    meta: path to meta data table (meta_open.csv)
    weather: path to weather data table (weatherX.csv)
    weather_file: name of the weather file
    industry: name of undustry to focus on
    OUTPUT
    A dataframe in which each record represents a building at a certain time
    columns =[
    building_name: name of the building from meta, str
    month: one-hot encoded
    day: from elec, int
    day_of_the_week: one-hot encoded
    hour: from elec, hour from weather is converted to the nearest :00, int
    area: from meta, float
    primary_space_usage: from meta (primaryspaceuse_abbrev), one-hot encoded
    electricity: from elec
    temperature: from weather
    ]
    ------------------------------------------------------------------------------------------------------------------------------------------
    comment:
    -humidity is sometimes missing in weather table
    '''
    #read tables
    elec = pd.read_csv(elec_path)
    meta = pd.read_csv(meta_path)
    weather = pd.read_csv(weather_path)

    #set 'uid' as index in meta
    meta = meta.set_index('uid')

    #parse date
    weather['timestamp'] = weather['timestamp'].apply(dateutil.parser.parse) #changing timestamp from string to date time
    elec['timestamp'] = elec['timestamp'].apply(dateutil.parser.parse) # ""

    #construct the dataframe to return
    # Use the inputted Industry to pull all buildings from that industry
    buildings = list(meta[(meta['newweatherfilename'] == weather_file) & (meta['industry']==industry)].index) 

    #Creat an empty dataframe with 5 columns
    df = pd.DataFrame(columns=['building_name', 'timestamp', 'electricity', 'area', 'primary_space_usage']) 

    #for now, this loop is fine, but maybe rewrite to make code faster/ look for function in pandas
    for building in buildings:
    #filename followed by [['column name']] selects specific columns in datafram
        subdf = elec[['timestamp', building]] #extracting the timestamp and electricity data from all buildings in electricity table
        subdf.columns = ['timestamp', 'electricity'] #naming columns
        subdf['building_name'] = building #non temporal column with building name, all same entry
        subdf['area'] = meta.loc[building, 'sqm'] #"" making new column called area
        subdf['primary_space_usage'] = meta.loc[building, 'primaryspaceuse_abbrev']
        df = pd.concat([df, subdf], axis=0, ignore_index=True) #combining the two tables. Where axis =0 puts the table under the first one, axis = 1 puts the table to the right 
        #df has 'building_name', timestamp, electricity meter, area, primary space usage
        # The timestamps are not matched up in the minutes between external and building data, cutoff the minutes and just match to hour 
        weather['rounded_timestamp'] = weather['timestamp'].apply(cutoff_minute) 
        #cutoff_minute is a function that is implemented below
    weather = weather.groupby('rounded_timestamp').first() #only the first observation in each hour is taken
    weather = weather['TemperatureC'] #only need temperature column

    df['timestamp'] = df['timestamp'].apply(cutoff_minute) #to drop timezone information
    df = df.join(weather, on='timestamp', how='inner', lsuffix='elec', rsuffix='weather') #join temperature data from weather table 
    #join aligns the tables by the timestamp index, instead of merely merging them, so that each instinance of a timestamp for each building has weather data

    #Adding columns for the month, year, date, hour, and weekday
    df['month']=df['timestamp'].apply(lambda x: x.month)
    df['year']=df['timestamp'].apply(lambda x: x.year)
    df['date']=df['timestamp'].apply(lambda x: x.day)
    df['hour']=df['timestamp'].apply(lambda x: x.hour)
    df['weekday']=df['timestamp'].apply(lambda x: x.dayofweek)

    #One-hot encode for month, year, date, hour, and weekday is pd.get_dummies
    #combining df with one-hot encodes the tables.  axis =1 puts the table under the first one, axis = 0 puts the table to the right 
    df.reset_index()
    df = pd.concat([df, pd.get_dummies(df['month'], prefix='month')],axis=1) 
    df = pd.concat([df, pd.get_dummies(df['date'], prefix='date')], axis=1) 
    df = pd.concat([df, pd.get_dummies(df['hour'], prefix='hour')], axis=1) 
    df = pd.concat([df, pd.get_dummies(df['weekday'], prefix='wkday')], axis=1) 
    df = pd.concat([df, pd.get_dummies(df['primary_space_usage'], prefix='PSU')], axis=1) 
    return df.reset_index(drop=True)

In [39]:
df = make_dataset(elec_path, meta_path, weather_path, 'weather1.csv', 'Education')
display(df)

c:\users\yalex\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\yalex\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\yalex\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

,area,building_name,electricity,primary_space_usage,timestamp,TemperatureC,month,year,date,hour,...,hour_23,wkday_0,wkday_1,wkday_2,wkday_3,wkday_4,wkday_5,wkday_6,PSU_PrimClass,PSU_UnivClass
0,2927.0,PrimClass_Jolie,0.700000,PrimClass,2014-12-01 00:00:00,7.0,12,2014,1,0,...,0,1,0,0,0,0,0,0,1,0
1,4373.0,PrimClass_Jaylin,1.100000,PrimClass,2014-12-01 00:00:00,7.0,12,2014,1,0,...,0,1,0,0,0,0,0,0,1,0
2,2132.0,PrimClass_Jayla,2.000000,PrimClass,2014-12-01 00:00:00,7.0,12,2014,1,0,...,0,1,0,0,0,0,0,0,1,0
3,2937.0,PrimClass_Janiya,2.300000,PrimClass,2014-12-01 00:00:00,7.0,12,2014,1,0,...,0,1,0,0,0,0,0,0,1,0
4,2152.0,PrimClass_Janice,1.400000,PrimClass,2014-12-01 00:00:00,7.0,12,2014,1,0,...,0,1,0,0,0,0,0,0,1,0
5,2572.0,PrimClass_Jaden,2.763000,PrimClass,2014-12-01 00:00:00,7.0,12,2014,1,0,...,0,1,0,0,0,0,0,0,1,0
6,2154.0,PrimClass_Jermaine,2.200000,PrimClass,2014-12-01 00:00:00,7.0,12,2014,1,0,...,0,1,0,0,0,0,0,0,1,0
7,3984.0,PrimClass_Josephine,4.700000,PrimClass,2014-12-01 00:00:00,7.0,12,2014,1,0,...,0,1,0,0,0,0,0,0,1,0
8,1877.0,PrimClass_Javier,2.900000,PrimClass,2014-12-01 00:00:00,7.0,12,2014,1,0,...,0,1,0,0,0,0,0,0,1,0
9,2939.0,PrimClass_Jaylinn,3.900000,PrimClass,2014-12-01 00:00:00,7.0,12,2014,1,0,...,0,1,0,0,0,0,0,0,1,0


In [40]:
def train_test(df, meta_path, weather_file, industry):
    meta = pd.read_csv(meta_path)
    #set 'uid' as index in meta
    meta = meta.set_index('uid')
    buildings = list(meta[(meta['newweatherfilename'] == weather_file) & (meta['industry']==industry)].index) 
    np.random.shuffle(buildings)
    train_buildings = buildings[:50]#take 50 buildings
    test_buildings = buildings[50:]#take the others
    df_train = df[df['building_name'].isin(train_buildings)]
    df_test = df[df['building_name'].isin(test_buildings)]
    df_train = df_train.sort_values(['building_name', 'timestamp']).reset_index(drop=True)
    df_test = df_test.sort_values(['building_name', 'timestamp']).reset_index(drop=True)
    df_train.to_csv('../data/weather1_education_train.csv')
    df_test.to_csv('../data/weather1_education_test.csv')

In [41]:
train_test(df, meta_path, 'weather1.csv', 'Education')

In [44]:
train = pd.read_csv('../data/weather1_education_train.csv')
train.head()

,Unnamed: 0,area,building_name,electricity,primary_space_usage,timestamp,TemperatureC,month,year,date,...,hour_23,wkday_0,wkday_1,wkday_2,wkday_3,wkday_4,wkday_5,wkday_6,PSU_PrimClass,PSU_UnivClass
0,0,2777.0,PrimClass_Jacqueline,5.0,PrimClass,2014-12-01 00:00:00,7.0,12,2014,1,...,0,1,0,0,0,0,0,0,1,0
1,1,2777.0,PrimClass_Jacqueline,5.2,PrimClass,2014-12-01 01:00:00,5.0,12,2014,1,...,0,1,0,0,0,0,0,0,1,0
2,2,2777.0,PrimClass_Jacqueline,5.2,PrimClass,2014-12-01 02:00:00,5.0,12,2014,1,...,0,1,0,0,0,0,0,0,1,0
3,3,2777.0,PrimClass_Jacqueline,5.5,PrimClass,2014-12-01 03:00:00,6.0,12,2014,1,...,0,1,0,0,0,0,0,0,1,0
4,4,2777.0,PrimClass_Jacqueline,6.0,PrimClass,2014-12-01 04:00:00,7.0,12,2014,1,...,0,1,0,0,0,0,0,0,1,0


In [45]:
train.shape

(437850, 88)

In [46]:
test = pd.read_csv('../data/weather1_education_test.csv')
test.head()

,Unnamed: 0,area,building_name,electricity,primary_space_usage,timestamp,TemperatureC,month,year,date,...,hour_23,wkday_0,wkday_1,wkday_2,wkday_3,wkday_4,wkday_5,wkday_6,PSU_PrimClass,PSU_UnivClass
0,0,6149.0,PrimClass_Jacob,34.900002,PrimClass,2014-12-01 00:00:00,7.0,12,2014,1,...,0,1,0,0,0,0,0,0,1,0
1,1,6149.0,PrimClass_Jacob,32.700000,PrimClass,2014-12-01 01:00:00,5.0,12,2014,1,...,0,1,0,0,0,0,0,0,1,0
2,2,6149.0,PrimClass_Jacob,32.200000,PrimClass,2014-12-01 02:00:00,5.0,12,2014,1,...,0,1,0,0,0,0,0,0,1,0
3,3,6149.0,PrimClass_Jacob,32.300000,PrimClass,2014-12-01 03:00:00,6.0,12,2014,1,...,0,1,0,0,0,0,0,0,1,0
4,4,6149.0,PrimClass_Jacob,32.599998,PrimClass,2014-12-01 04:00:00,7.0,12,2014,1,...,0,1,0,0,0,0,0,0,1,0


In [47]:
test.shape

(166383, 88)

In [48]:
train['building_name'].unique().shape

(50,)

In [49]:
test['building_name'].unique().shape

(19,)